In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json, datetime

In [2]:
root_path = ''

with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [5]:
query = f"""
SELECT 
    item.ItemLookupCode, 
    item.Quantity,
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND Item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [8]:
fromPOS['Cost']= fromPOS[['itemCost', 'spCost']].apply(max, axis=1)
low_price = fromPOS[fromPOS['Price']<fromPOS['Cost']*1.5]

In [10]:
low_price.to_excel('low_price.xlsx', index=False)

: 